In [2]:
import numpy as np
import cv2
from keras import backend as K

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, Input, MaxPooling2D, AveragePooling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
import keras.layers
from keras.optimizers import RMSprop

import matplotlib.pyplot as plt
%matplotlib

Using TensorFlow backend.


Using matplotlib backend: Qt5Agg


In [2]:
decoder = Sequential()
depth = 32
dropout = 0.4
input_shape = (1, 256, 256)
decoder.add(Conv2D(depth, 3, input_shape=input_shape, padding='same'))
decoder.add(BatchNormalization(momentum=0.9))
decoder.add(LeakyReLU(alpha=0.2))

decoder.add(Conv2D(depth*2, 3, padding='same'))
decoder.add(BatchNormalization(momentum=0.9))
decoder.add(LeakyReLU(alpha=0.2))

decoder.add(Conv2D(depth*4, 3, padding='same'))
decoder.add(BatchNormalization(momentum=0.9))
decoder.add(LeakyReLU(alpha=0.2))

decoder.add(Conv2D(depth*2, 3, padding='same'))
decoder.add(BatchNormalization(momentum=0.9))
decoder.add(LeakyReLU(alpha=0.2))

decoder.add(Conv2D(depth, 3, padding='same'))
decoder.add(BatchNormalization(momentum=0.9))
decoder.add(LeakyReLU(alpha=0.2))

decoder.add(Conv2D(1, 1, padding='same', activation='sigmoid'))

decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 256, 256)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 256, 256)      1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 256, 256)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 256, 256)      18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 256, 256)      1024      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 256, 256)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 256, 256)     73856     
__________

In [3]:
def InceptionBlock(filters_in, filters_out):
    input_layer = Input(shape=(filters_in, 256, 256))
    tower_filters = int(filters_out / 4)

    tower_1 = Conv2D(tower_filters, 1, padding='same', activation='relu')(input_layer)

    tower_2 = Conv2D(tower_filters, 1, padding='same', activation='relu')(input_layer)
    tower_2 = Conv2D(tower_filters, 3, padding='same', activation='relu')(tower_2)

    tower_3 = Conv2D(tower_filters, 1, padding='same', activation='relu')(input_layer)
    tower_3 = Conv2D(tower_filters, 5, padding='same', activation='relu')(tower_3)

    tower_4 = MaxPooling2D(tower_filters, padding='same', strides=(1, 1))(input_layer)
    tower_4 = Conv2D(tower_filters, 1, padding='same', activation='relu')(tower_4)

    concat = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis=1)

    res_link = Conv2D(filters_out, 1, padding='same', activation='relu')(input_layer)

    output = keras.layers.add([concat, res_link])
    output = Activation('relu')(output)

    model_output = Model([input_layer], output)
    return model_output

In [4]:
encoder = Sequential()
input_shape = (2, 256, 256)

encoder.add(Conv2D(16, 3, input_shape=input_shape, padding='same'))
encoder.add(BatchNormalization(momentum=0.9))
encoder.add(LeakyReLU(alpha=0.2))

encoder.add(InceptionBlock(16, 32))    # Layer 2
encoder.add(InceptionBlock(32, 64))    # Layer 3
encoder.add(InceptionBlock(64, 128))   # Layer 4
encoder.add(InceptionBlock(128, 256))  # Layer 5
encoder.add(InceptionBlock(256, 128))  # Layer 6
encoder.add(InceptionBlock(128, 64))   # Layer 7
encoder.add(InceptionBlock(64, 32))    # Layer 8

encoder.add(Conv2D(16, 3, padding='same')) # Layer 9
encoder.add(BatchNormalization(momentum=0.9))
encoder.add(LeakyReLU(alpha=0.2))

encoder.add(Conv2D(1, 1, padding='same', activation='tanh')) # Output
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 16, 256, 256)      304       
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 256, 256)      1024      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 16, 256, 256)      0         
_________________________________________________________________
model_1 (Model)              (None, 32, 256, 256)      3280      
_________________________________________________________________
model_2 (Model)              (None, 64, 256, 256)      12960     
_________________________________________________________________
model_3 (Model)              (None, 128, 256, 256)     51520     
_________________________________________________________________
model_4 (Model)              (None, 256, 256, 256)     205440    
__________

In [7]:
from SpatialPyramidPooling import SpatialPyramidPooling

discriminator = Sequential()
input_shape = (3, 256, 256)

discriminator.add(Conv2D(8, 3, padding='same', input_shape=input_shape))
discriminator.add(BatchNormalization(momentum=0.9))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(AveragePooling2D(pool_size=5, strides=2, padding='same'))

discriminator.add(Conv2D(16, 3, padding='same'))
discriminator.add(BatchNormalization(momentum=0.9))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(AveragePooling2D(pool_size=5, strides=2, padding='same'))
                  
discriminator.add(Conv2D(32, 1, padding='valid'))
discriminator.add(BatchNormalization(momentum=0.9))
discriminator.add(AveragePooling2D(pool_size=5, strides=2, padding='same'))

discriminator.add(Conv2D(64, 1, padding='valid'))
discriminator.add(BatchNormalization(momentum=0.9))
discriminator.add(AveragePooling2D(pool_size=5, strides=2, padding='same'))

discriminator.add(Conv2D(128, 3, padding='same'))
discriminator.add(BatchNormalization(momentum=0.9))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(AveragePooling2D(pool_size=5, strides=2, padding='same'))

discriminator.add(SpatialPyramidPooling([1, 2, 4]))

discriminator.add(Dense(128))

discriminator.add(Dense(2, activation='tanh'))

discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           (None, 8, 256, 256)       224       
_________________________________________________________________
batch_normalization_8 (Batch (None, 8, 256, 256)       1024      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 8, 256, 256)       0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 8, 128, 128)       0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 16, 128, 128)      1168      
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 128, 128)      512       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 16, 128, 128)      0         
__________

In [6]:
from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(color=True, resize=1.0, slice_=(slice(0, 250), slice(0, 250)))

In [8]:
images_rgb = lfw_people.images
images_rgb = np.moveaxis(images_rgb, -1, 1)
print(images_rgb.shape)

(13233, 3, 250, 250)


In [9]:
def rgb2ycc(img_rgb):
    """
    Takes as input a RGB image and convert it to Y Cb Cr space. The shape is channels first.
    """
    output = np.zeros(np.shape(img_rgb))
    output[0, :, :] = 0.299 * img_rgb[0, :, :] + 0.587 * img_rgb[1, :, :] + 0.114 * img_rgb[2, :, :]
    output[1, :, :] = -0.1687 * img_rgb[0, :, :] - 0.3313 * img_rgb[1, :, :] + 0.5 * img_rgb[2, :, :] + 128
    output[2, :, :] = 0.5 * img_rgb[0, :, :] - 0.4187 * img_rgb[1, :, :] + 0.0813 * img_rgb[2, :, :] + 128

    return output

In [10]:
images_ycc = np.zeros(images_rgb.shape)
for k in range(images_rgb.shape[0]):
    images_ycc[k, :, :, :] = rgb2ycc(images_rgb[k, :, :, :])

In [11]:
X_train = (images_ycc.astype(np.float32) - 127.5) / 127.5

In [12]:
batch_size = 128
cover_idx = np.random.randint(0, X_train.shape[0], batch_size)
secret_idx = np.random.randint(0, X_train.shape[0], batch_size)
cover_imgs = X_train[cover_idx]
cover_imgs_Y = cover_
secret_imgs = X_train[secret_idx]

enc_input = np.concatenate((cover_imgs, secret_imgs), axis=1)
# enc_imgs = self.encoder.predict(enc_input)

In [13]:
def rgb2gray(img_rgb):
    """
    Transform a RGB image into a grayscale one using weighted method. Shape: channels first.
    """
    output = np.zeros((1, img_rgb.shape[1], img_rgb.shape[2]))
    output[0, :, :] = 0.3 * img_rgb[0, :, :] + 0.59 * img_rgb[1, :, :] + 0.11 * img_rgb[2, :, :]
    return output

In [17]:
images_gray = np.zeros((images_rgb.shape[0], 1, images_rgb.shape[2], images_rgb.shape[3]))
for k in range(images_rgb.shape[0]):
    images_gray[k, 0, :, :] = rgb2gray(images_rgb[k, :, :, :])

In [15]:
A = np.random.rand(3,4,4,4)
print(A[0,0,:,:])
B = np.pad(A, ((0,0), (0,0), (3,3), (3,3)), 'constant')
print(B[0,0,:,:])

[[ 0.10833984  0.95407497  0.78329048  0.71951861]
 [ 0.10182542  0.49493423  0.45324552  0.64322044]
 [ 0.88850775  0.37323487  0.54702865  0.90627754]
 [ 0.87230472  0.25230752  0.51192716  0.24997722]]
[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.10833984  0.95407497  0.78329048
   0.71951861  0.          0.          0.        ]
 [ 0.          0.          0.          0.10182542  0.49493423  0.45324552
   0.64322044  0.          0.          0.        ]
 [ 0.          0.          0.          0.88850775  0.37323487  0.54702865
   0.90627754  0.          0.          0.        ]
 [ 0.          0.          0.          0.87230472  0.25230752  0.5119